
# Лабораторная работа 5

## Выполнил: Морозов Алексей, М32071

## Вариант: 9, задание: 2


In [1]:
import numpy as np
import scipy as scp
import matplotlib as mlt
import matplotlib.pyplot as plt
import pandas as pd

import sklearn.linear_model as lm

import matplotlib.axes._axes as axes
import matplotlib.figure as figure


## Условие

В файле $mobile\_phones.cvs$ представлены данные о мобильных телефонах

1. Построить линейную модель, где в качестве независимых переменных выступают высота, ширина экрана (атрибуты $sc\_h$ и $sc\_w$) и емкость аккумулятора (вместе со свободным коэффициентом), зависимой – масса телефона, вычислить оценки коэффициентов модели и остаточной дисперсии, построить для них доверительные интервалы, вычислить коэффициент детерминации
2. Проверить следующие подозрения:
    * Чем больше высота экрана, тем больше масса
    * Чем больше ширина экрана, тем больше масса
    * Масса зависит и от ширины экрана, и от емкости аккумулятора


## Решение

### Построение модели


Введем константу $u = 1$

In [2]:
df_all = pd.read_csv('mobile_phones.csv')[['sc_h', 'sc_w', 'battery_power', 'mobile_wt']]
df_all['u'] = 1
df_all

,sc_h,sc_w,battery_power,mobile_wt,u
0,9,7,842,188,1
1,17,3,1021,136,1
2,11,2,563,145,1
3,16,8,615,131,1
4,8,2,1821,141,1
...,...,...,...,...,...
1995,13,4,794,106,1
1996,11,10,1965,187,1
1997,9,1,1911,108,1
1998,18,10,1512,145,1


Уравнение регрессии $4$ параметров имеет вид
\begin{equation}
    f = ax + by + cz + du = ax + by + cz + d
\end{equation}
Найдем коэффициенты методом наименьших квадратов

In [3]:
x_data = df_all[['sc_h', 'sc_w', 'battery_power', 'u']]
y_data = df_all['mobile_wt']
coef = scp.optimize.lsq_linear(x_data, y_data).x
coef

array([-2.63548169e-01, -3.95486159e-02,  6.44803879e-05,  1.43640572e+02])

Полученное уравнение

In [4]:
print(f'f = {coef[0]}x + {coef[1]}y + {coef[2]}z + {coef[3]}')

f = -0.2635481693123999x + -0.03954861594447834y + 6.448038790873476e-05z + 143.64057226048243


In [5]:
fred_deg = len(x_data) - len(coef)

In [6]:
ssr = np.sum((y_data - x_data @ coef)**2)
var_resid = ssr / fred_deg
print('Остаточная дисперсия', var_resid)

Остаточная дисперсия 1253.557630585452


In [7]:
r_sq = 1 - var_resid / y_data.var()
print('Коэффициент детерминации', r_sq)

Коэффициент детерминации -0.0003368061790351984


Для получения дисперсий коэффициентов и построения для них доверительных интервалов построим матрицу ковариаций, диагональные элементы которой равны дисперсии $i$-ого коэффициента

In [8]:
cov_mat = var_resid * np.linalg.inv(x_data.T @ x_data)
cov_mat

array([[ 4.75165494e-02, -2.32411013e-02,  8.71347946e-06,
        -4.61522790e-01],
       [-2.32411013e-02,  4.44255349e-02,  2.75853091e-06,
         2.63980615e-02],
       [ 8.71347946e-06,  2.75853091e-06,  3.25078512e-06,
        -4.14929839e-03],
       [-4.61522790e-01,  2.63980615e-02, -4.14929839e-03,
         1.12932542e+01]])

In [9]:
print('Доверительные интервалы коэффициентов')
for i in range(len(coef)):
    print(f'{i}: {coef[i] - scp.stats.t.isf(0.05 / 2, fred_deg) * np.sqrt(cov_mat[i][i])} < {coef[i]} < {coef[i] + scp.stats.t.isf(0.05 / 2, fred_deg) * np.sqrt(cov_mat[i][i])}')

Доверительные интервалы коэффициентов
0: -0.6910460538769582 < -0.2635481693123999 < 0.1639497152521584
1: -0.45290805116435595 < -0.03954861594447834 < 0.3738108192753993
2: -0.003471465855209316 < 6.448038790873476e-05 < 0.0036004266310267854
3: 137.05003112925993 < 143.64057226048243 < 150.23111339170492


In [10]:
print('Доверительный интервал остаточной дисперсии')
print(f'{ssr / scp.stats.chi2.isf(0.05 / 2, fred_deg)} < {var_resid} < {ssr / scp.stats.chi2.ppf(0.05 / 2, fred_deg)}')

Доверительный интервал остаточной дисперсии
1179.2805226501582 < 1253.557630585452 < 1335.1160518152021



### Проверка гипотез


Используем $F$-критерий

\begin{gather}
    H_0: \text{a=0, масса не зависит от высоты экрана} \\
    H_1: \neg{H_0}
\end{gather}

In [20]:
x_data_r = df_all[['sc_w', 'battery_power', 'u']]
res_r = scp.optimize.lsq_linear(x_data_r, y_data)
coef_r = res_r.x

ssr_r = np.sum((y_data - x_data_r @ coef_r)**2)
f = ((ssr_r - ssr) / ssr) * fred_deg / 1
p = 1 - scp.stats.f.cdf(f, 1, fred_deg)
print('p-value', p)

p-value 0.22679400279494932


\begin{gather}
    H_0: \text{b=0, масса не зависит от ширины экрана} \\
    H_1: \neg{H_0}
\end{gather}

In [21]:
x_data_r = df_all[['sc_h', 'battery_power', 'u']]
res_r = scp.optimize.lsq_linear(x_data_r, y_data)
coef_r = res_r.x

ssr_r = np.sum((y_data - x_data_r @ coef_r) ** 2)
f = ((ssr_r - ssr) / ssr) * fred_deg / 1
p = 1 - scp.stats.f.cdf(f, 1, fred_deg)
print('p-value', p)

p-value 0.851181484918942


\begin{gather}
    H_0: \text{b=c=0, масса не зависит от ширины и емкости аккумулятора} \\
    H_1: \neg{H_0}
\end{gather}

In [22]:
x_data_r = df_all[['sc_h', 'u']]
res_r = scp.optimize.lsq_linear(x_data_r, y_data)
coef_r = res_r.x

ssr_r = np.sum((y_data - x_data_r @ coef_r) ** 2)
f = ((ssr_r - ssr) / ssr) * fred_deg / 2
p = 1 - scp.stats.f.cdf(f, 2, fred_deg)
print('p-value', p)

p-value 0.9818737559869272


Во всех случаях значение $p-value$ больше уровня значимости $\alpha=0.05$, поэтому все три гипотезы принимаются

In [23]:
x_data_r1 = df_all[['sc_h', 'battery_power', 'u']]
coef_r1 = scp.optimize.lsq_linear(x_data_r1, y_data).x

ssr_r1 = np.sum((y_data - x_data_r1 @ coef_r1) ** 2)
f1 = ((ssr_r1 - ssr) / ssr) * fred_deg / 1
p1 = 1 - scp.stats.f.cdf(f1, 1, fred_deg)

x_data_r2 = df_all[['sc_h', 'sc_w', 'u']]
coef_r2 = scp.optimize.lsq_linear(x_data_r2, y_data).x

ssr_r2 = np.sum((y_data - x_data_r2 @ coef_r2) ** 2)
f2 = ((ssr_r2 - ssr) / ssr) * fred_deg / 1
p2 = 1 - scp.stats.f.cdf(f2, 1, fred_deg)

print('p-value', min(p1, p2))

p-value 0.851181484918942
